In [57]:
import pandas as pd

In [58]:
columns = ['col1', 'col2', 'col3', 'col4', 'col5']

In [59]:
def merge_columns(row):
    return ", ".join(
            dict.fromkeys(
                str(x) for x in row if pd.notna(x) and str(x).strip() != ""
            )
        )

In [ ]:
expected = pd.read_csv("data_poleval/dev-0/expected.tsv", header=None, sep='\t', names=columns)

expected["answer"] = (expected.apply(merge_columns,axis=1))

expected = expected[['answer']]

In [ ]:
questions = pd.read_csv("data_poleval/dev-0/in.tsv", header=None, names=["col1", "col2", "col3", "col4"])

expected["question"] = (questions.apply(merge_columns,axis=1))

In [62]:
pd.set_option("display.max_colwidth", None)

In [63]:
expected["document"] = expected.apply(lambda row: f"{row["question"]} -> {row["answer"]}", axis=1)

In [69]:
expected

,answer,question,document
0,alfa,Jak nazywa się pierwsza litera alfabetu greckiego?,Jak nazywa się pierwsza litera alfabetu greckiego? -> alfa
1,cięciwa,Jak nazywa się dowolny odcinek łączący dwa punkty okręgu?,Jak nazywa się dowolny odcinek łączący dwa punkty okręgu? -> cięciwa
2,w Egipcie,W którym państwie rozpoczyna się akcja powieści „W pustyni i w puszczy”?,W którym państwie rozpoczyna się akcja powieści „W pustyni i w puszczy”? -> w Egipcie
3,tak,Czy w państwach starożytnych powoływani byli posłowie i poselstwa?,Czy w państwach starożytnych powoływani byli posłowie i poselstwa? -> tak
4,Mazowsze,W jakim zespole występowała Hanka w filmie „Żona dla Australijczyka”?,W jakim zespole występowała Hanka w filmie „Żona dla Australijczyka”? -> Mazowsze
...,...,...,...
995,tak,Czy Antonín Dvořák żył w XX wieku?,Czy Antonín Dvořák żył w XX wieku? -> tak
996,ptasiego,"Jakiego mleka brakuje osobie, która ma wszystko, czego zapragnie?","Jakiego mleka brakuje osobie, która ma wszystko, czego zapragnie? -> ptasiego"
997,"napęd na cztery koła, napęd na 4 koła, four wheel drive",Co oznacza skrót 4WD na samochodach?,"Co oznacza skrót 4WD na samochodach? -> napęd na cztery koła, napęd na 4 koła, four wheel drive"
998,w warmińsko-mazurskim,W którym województwie leży Giżycko?,W którym województwie leży Giżycko? -> w warmińsko-mazurskim


In [64]:
documents_v1 = expected["document"].values

TF-IDF

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(
            max_features=5000,
            ngram_range=(1,3),
            min_df=2,
            max_df=0.8,
            smooth_idf=True,
            sublinear_tf=True)

matrix = tf_idf.fit_transform(documents_v1)

In [66]:
from sklearn.metrics.pairwise import cosine_similarity

def get_nearest_documents(query, model, matrix, documents, top_k=5):
    query_vec = model.transform([query])

    similarities = cosine_similarity(query_vec, matrix).flatten()

    top_doc = similarities.argsort()[::-1][:top_k]

    docs = []
    for i in top_doc:
        docs.append(documents[i])

    return docs

In [68]:
get_nearest_documents(query="Na którym kontynencie leżała Kartagina?",
                      model=tf_idf,
                      matrix=matrix,
                      documents=documents_v1)

['Na którym kontynencie rządził Mobutu Sese Seko? -> w Afryce',
 'Na którym kontynencie występuje ryś iberyjski? -> w Europie',
 'Na którym kontynencie znajduje się pustynia Takla Makan? -> w Azji',
 'Na którym kontynencie żyją Irokezi? -> w Ameryce Północnej',
 'Na którym kontynencie leży Gabon? -> w Afryce']